In [11]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import psutil
import plotly.graph_objs as go
import datetime
import pandas as pd

# INITIALIZE THE CODE
app = dash.Dash(__name__)
server = app.server  # Fix potential deployment issues

#LAYOUT
app.layout = html.Div([
    html.H1("Real-Time System Monitoring Dashboard"),
    
    #CPU USAGE GRAPH
    dcc.Graph(id='cpu-usage-graph'),
    
    # MEMORY USAGE
    dcc.Graph(id='memory-usage-graph'),
    
    # DISK USAGE
    dcc.Graph(id='disk-usage-graph'),
    
    # PROCESS TABLE
    html.H3("Running Processes"),
    dash.dash_table.DataTable(
        id='process-table',
        columns=[
            {'name': 'PID', 'id': 'pid'},
            {'name': 'Name', 'id': 'name'},
            {'name': 'CPU %', 'id': 'cpu_percent'},
            {'name': 'Memory %', 'id': 'memory_percent'}
        ],
        style_table={'overflowX': 'auto'},
        style_header={'backgroundColor': 'black', 'color': 'white'},
        style_cell={'textAlign': 'left'},
    ),
    
    #AUTO REFRESH INTERVAL
    dcc.Interval(
        id='interval-component',
        interval=2000,  # Update every 2 seconds
        n_intervals=0
    )
])

# STORE HISTORICAL CPU USAGE
cpu_history = []

# UPDATE CPU USAGE GRAPH
@app.callback(
    Output('cpu-usage-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_cpu_graph(n):
    global cpu_history
    timestamp = datetime.datetime.now().strftime('%H:%M:%S')
    cpu_percent = psutil.cpu_percent(interval=None)
    cpu_history.append((timestamp, cpu_percent))
    if len(cpu_history) > 20:
        cpu_history.pop(0)
    
    x_vals, y_vals = zip(*cpu_history)
    figure = {
        'data': [go.Scatter(x=list(x_vals), y=list(y_vals), mode='lines+markers')],
        'layout': {'title': 'CPU Usage (%)', 'xaxis': {'title': 'Time'}, 'yaxis': {'title': 'Percentage'}}
    }
    return figure

# UPDATE MEMORY USAGE GRAPH
@app.callback(
    Output('memory-usage-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_memory_graph(n):
    mem = psutil.virtual_memory()
    figure = {
        'data': [go.Pie(labels=['Used', 'Available'], values=[mem.used, mem.available])],
        'layout': {'title': 'Memory Usage'}
    }
    return figure

# UPDATE DISK USAGE GRAPH
@app.callback(
    Output('disk-usage-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_disk_graph(n):
    disk = psutil.disk_usage('/')
    figure = {
        'data': [go.Pie(labels=['Used', 'Free'], values=[disk.used, disk.free])],
        'layout': {'title': 'Disk Usage'}
    }
    return figure

# UPDATE PROCESS TABLE
@app.callback(
    Output('process-table', 'data'),
    [Input('interval-component', 'n_intervals')]
)
def update_process_table(n):
    processes = []
    for proc in psutil.process_iter(attrs=['pid', 'name', 'cpu_percent', 'memory_percent']):
        try:
            processes.append(proc.info)
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            continue
    df = pd.DataFrame(processes)
    if not df.empty:
        df = df.nlargest(10, 'cpu_percent')  # Show top 10 CPU consuming processes
    return df.to_dict('records')

if __name__ == '__main__':
    app.run(debug=True, port=8060)  # Specify an alternative port to avoid conflicts


In [12]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import psutil
import plotly.graph_objs as go
import datetime
import pandas as pd

# Initialize the Dash app
app = dash.Dash(__name__)
server = app.server  # Fix potential deployment issues

# Layout
app.layout = html.Div([
    html.H1("Real-Time System Monitoring Dashboard"),
    
    # CPU Usage Graph
    dcc.Graph(id='cpu-usage-graph'),
    
    # Memory Usage
    dcc.Graph(id='memory-usage-graph'),
    
    # Disk Usage
    dcc.Graph(id='disk-usage-graph'),
    
    # Process Table
    html.H3("Running Processes"),
    dash.dash_table.DataTable(
        id='process-table',
        columns=[
            {'name': 'PID', 'id': 'pid'},
            {'name': 'Name', 'id': 'name'},
            {'name': 'CPU %', 'id': 'cpu_percent'},
            {'name': 'Memory %', 'id': 'memory_percent'}
        ],
        style_table={'overflowX': 'auto'},
        style_header={'backgroundColor': 'black', 'color': 'white'},
        style_cell={'textAlign': 'left'},
    ),
    
    # Auto Refresh Interval
    dcc.Interval(
        id='interval-component',
        interval=2000,  # Update every 2 seconds
        n_intervals=0
    )
])

# Store historical CPU usage
cpu_history = []

# Update CPU Usage Graph
@app.callback(
    Output('cpu-usage-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_cpu_graph(n):
    global cpu_history
    timestamp = datetime.datetime.now().strftime('%H:%M:%S')
    cpu_percent = psutil.cpu_percent(interval=None)
    cpu_history.append((timestamp, cpu_percent))
    if len(cpu_history) > 20:
        cpu_history.pop(0)
    
    x_vals, y_vals = zip(*cpu_history)
    figure = {
        'data': [go.Scatter(x=list(x_vals), y=list(y_vals), mode='lines+markers')],
        'layout': {'title': 'CPU Usage (%)', 'xaxis': {'title': 'Time'}, 'yaxis': {'title': 'Percentage'}}
    }
    return figure

# Update Memory Usage Graph
@app.callback(
    Output('memory-usage-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_memory_graph(n):
    mem = psutil.virtual_memory()
    figure = {
        'data': [go.Pie(labels=['Used', 'Available'], values=[mem.used, mem.available])],
        'layout': {'title': 'Memory Usage'}
    }
    return figure

# Update Disk Usage Graph
@app.callback(
    Output('disk-usage-graph', 'figure'),
    [Input('interval-component', 'n_intervals')]
)
def update_disk_graph(n):
    disk = psutil.disk_usage('/')
    figure = {
        'data': [go.Pie(labels=['Used', 'Free'], values=[disk.used, disk.free])],
        'layout': {'title': 'Disk Usage'}
    }
    return figure

# Update Process Table
@app.callback(
    Output('process-table', 'data'),
    [Input('interval-component', 'n_intervals')]
)
def update_process_table(n):
    processes = []
    for proc in psutil.process_iter(attrs=['pid', 'name', 'cpu_percent', 'memory_percent']):
        try:
            processes.append(proc.info)
        except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
            continue
    df = pd.DataFrame(processes)
    if not df.empty:
        df = df.nlargest(10, 'cpu_percent')  # Show top 10 CPU consuming processes
    return df.to_dict('records')

if __name__ == '__main__':
    app.run(debug=True, port=8060)  # Specify an alternative port to avoid conflicts
